# Mortgage Risk Analyzer v5
created Oct 2022

## Intro
I created this document to help myself and others realized the financtal implications of buying a home.
Adjust the sliders and see the inpact on the output below

In [1]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:1000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
.jp-OutputArea {
    overflow-y: unset !important;
}
.widget-box {
    overflow: unset !important;
}
</style>

In [2]:
import ipywidgets as widgets
width='95%'
input_sliders = []
input_sliders.append(widgets.FloatSlider(value=3,min=1,max=10,step=0.01,description='Intrest Rate (%/yr)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
input_sliders.append(widgets.IntSlider(value=300,min=100,max=1000,description='House Price ($k)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
input_sliders.append(widgets.IntSlider(value=60,min=1,max=100,description='Down Payment ($k)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
input_sliders.append(widgets.IntSlider(value=30,min=1,max=60,description='Term (years)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
input_sliders.append(widgets.FloatSlider(value=2,min=0,max=5,description='Private Mortgage Insurance (%/yr)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
input_sliders.append(widgets.FloatSlider(value=1,min=1,max=5,description='Payment Multiplier (%)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
input_sliders.append(widgets.IntSlider(value=1500,min=500,max=5000,description='Specific Payment Amount ($)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
input_radios = widgets.RadioButtons(
    options=['Use Payment Multiplier', 'Use Specific Payment Amount'],
    description="Payment Choice"
)

onetime_sliders = []
onetime_sliders.append(widgets.FloatSlider(value=2,min=1,max=5,description='Closing Costs (%)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
onetime_sliders.append(widgets.IntSlider(value=350,min=300,max=400,description='Appraisal Fee ($)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
onetime_sliders.append(widgets.IntSlider(value=400,min=300,max=500,description='Inspection Fee ($)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
onetime_sliders.append(widgets.IntSlider(value=0,min=0,max=5000,description='Paint ($)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
onetime_sliders.append(widgets.IntSlider(value=0,min=0,max=5000,description='Cabinets ($)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
onetime_sliders.append(widgets.IntSlider(value=0,min=0,max=5000,description='Appliances ($)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
onetime_sliders.append(widgets.IntSlider(value=0,min=0,max=5000,description='Furniture ($)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
onetime_sliders.append(widgets.IntSlider(value=0,min=0,max=5000,description='Windows ($)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
onetime_sliders.append(widgets.IntSlider(value=0,min=0,max=5000,description='Roof ($)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
onetime_sliders.append(widgets.IntSlider(value=0,min=0,max=5000,description='Tile ($)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
onetime_sliders.append(widgets.IntSlider(value=0,min=0,max=5000,description='Carpet ($)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
onetime_sliders.append(widgets.IntSlider(value=0,min=0,max=5000,description='Lawn ($)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
onetime_sliders.append(widgets.IntSlider(value=0,min=0,max=5000,description='Doors ($)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))


lifetime_sliders = []
lifetime_sliders.append(widgets.IntSlider(value=600,min=500,max=5000,description='Home Insurance ($/yr)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
lifetime_sliders.append(widgets.IntSlider(value=200,min=0,max=700,description='HOA contribution ($/mo)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
lifetime_sliders.append(widgets.FloatSlider(value=4.5,min=0,max=200,description='Property Taxes ($k/yr)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))
lifetime_sliders.append(widgets.IntSlider(value=300,min=0,max=1000,description='Utilities ($/mo)',style={'description_width': 'initial'},layout=widgets.Layout(width=width)))


In [3]:
def calculateX(intrestRate, housePrice, downPayment, term, privateMortgageInsurancePct=0, minPaymentMuliplyer=1, specificPaymentAmount=None):
    intrestRate = intrestRate/100
    housePrice = housePrice*1000
    downPayment = downPayment*1000
    privateMortgageInsurancePct = privateMortgageInsurancePct/100/12 # monthly pmi
    
    #print('Calculating x for ',intrestRate,' ',housePrice,' ',downPayment,' ',term,' ',minPaymentMuliplyer)
    n = 12*term
    monthlyInterestRate = intrestRate / 12
    #print("monthlyInterestRate =",monthlyInterestRate)
    r = 1/(1+monthlyInterestRate)
    loan = housePrice - downPayment
    #print("loan =",loan)
    minPayment = loan * ((1-r)/(r-r**(n+1)))
    #print("minpayment =",minPayment)
    payment = minPayment * minPaymentMuliplyer
    if specificPaymentAmount:
        payment = specificPaymentAmount
    #print("payment =",payment)
    
    paymentSum = 0
    totalmonths = 0
    totalmonthsWithPMI = 0
    totalprivateMortgageInsurance = 0
    balance = loan
    
    while balance > 1:
        #print('balance ',balance)
        # 1 month passes
        interest = balance * monthlyInterestRate
        #print('interest ',interest)
        
        p = payment
        #print('payment ',p)
        if interest + balance < payment:
            p = interest + balance
        
        privateMortgageInsurance = 0
        if ((paymentSum + downPayment) < loan * 0.2) and (totalmonths < term*12/2):
            privateMortgageInsurance = privateMortgageInsurancePct*loan
            totalmonthsWithPMI += 1
            totalprivateMortgageInsurance += privateMortgageInsurance
        
        if p < interest:
            #print('error. payment too small ',intrestRate,' ',housePrice,' ',downPayment,' ',term,' ',minPaymentMuliplyer)
            return 0,0,0,0,0,0
            
        balance = balance - (p) + (interest)
        paymentSum += p
        totalmonths += 1
    expenses = paymentSum + downPayment + totalprivateMortgageInsurance
    x = expenses / housePrice
    return x, totalmonths, minPayment, payment, totalmonthsWithPMI, totalprivateMortgageInsurance
    
        

In [4]:
def calculateOneTimeExpenses(housePrice, downPayment, closingCostsPrct=0, appraisalFee=0, inspectionFee=0):
    housePrice = housePrice*1000
    downPayment = downPayment*1000
    closingCostsPrct = closingCostsPrct/100
    loan = housePrice - downPayment
    closingCosts = closingCostsPrct*loan
    return closingCosts + appraisalFee + inspectionFee

In [5]:
def calculateLifetimeExpensesPerMonth(homeInsurance,HOA,propertyTaxes,utilities):
    return homeInsurance/12 + HOA + propertyTaxes*1000/12 + utilities


In [6]:
def input_slider_values():
    values = []
    for s in input_sliders:
        values.append(s.value)
    return values
def onetime_slider_values():
    values = []
    for s in onetime_sliders:
        values.append(s.value) 
    return values
def lifetime_slider_values():
    values = []
    for s in lifetime_sliders:
        values.append(s.value)
    return values


In [7]:
import matplotlib.pyplot as plt
%matplotlib inline
def mccgraph(x,y,xlabel):
    #plt.figure()
    plt.xlabel(xlabel)
    plt.ylabel('x')
    plt.plot(x, y, '-ok')
    plt.show()

In [8]:
def show_comparitive_graphs(v0,v1,v2,v3,v4,v5,v6):
    variables = []
    for s in input_sliders:
        smin = int(s.min*100)
        smax = int(s.max*100)
        diff = smax-smin
        points_per_graph = 20
        step = int(diff/points_per_graph)
        variables.append([x / 100 for x in range(smin, smax, step)])
    for i,var in enumerate(variables):
        #print(var)
        values = input_slider_values()
        output = []
        xlabel = ''
        xvalues = []
        for v in var:
            #print(v)
            values[i] = v
            xlabel = input_sliders[i].description
            x, totalmonths, minPayment, payment, totalmonthsWithPMI, totalPMI = calculateX(values[0],values[1],values[2],values[3],values[4],values[5],values[6])
            if x>0:
                output.append(x)
                xvalues.append(v)
        mccgraph(xvalues, output, xlabel)
    #plt.show()

In [9]:
def show_output1(v0,v1,v2,v3,v4,v5,v6,r1,o0,o1,o2,o3,o4,o5,o6,o7,o8,o9,o10,o11,o12,l0,l1,l2,l3):
    houseprice = v1
    if (r1 == 'Use Payment Multiplier'):
        v6 = None
    else:
        v5 = 1
    print("House Price: "+int_to_string(houseprice)+"k")
    x, totalmonths, minPayment, payment, totalmonthsWithPMI, totalPMI = calculateX(v0,v1,v2,v3,v4,v5,v6)
    print("Minimum monthly payment = "+int_to_string(minPayment)+" and your chosen monthly payment = "+int_to_string(payment))
    print("You will spend "+int_to_string(x*houseprice)+"k on your mortgage in "+str(round(totalmonths/12,2))+" years (x="+str(round(x,2))+")")
    print("Months paid with PMI = "+str(round(totalmonthsWithPMI,2))+" months and total PMI amount = "+int_to_string(totalPMI))
    onetime_total = int(houseprice*(o0/100)*1000 + o1 + o2 + o3 + o4 + o5 + o6 + o7 + o8 + o9 + o10 + o11 + o12)
    print("You will have a one-time payment of "+int_to_string(onetime_total))
    totalPrice = int(x*houseprice*1000) + onetime_total
    print("The total price of your house including mortgage and one-time costs is "+int_to_string(totalPrice))
    lifetime_total = l0 + l1*12 + l2*1000 + l3*12
    print("Additionally every year you will pay "+int_to_string(lifetime_total)+"/yr")
def show_output2(v0,v1,v2,v3,v4,v5,v6):
    show_comparitive_graphs(v0,v1,v2,v3,v4,v5,v6)
    

def int_to_string(v):
    return '${:,.0f}'.format(v)

interactive_plot1= widgets.interactive_output(show_output1, {
 'v0':input_sliders[0]
, 'v1':input_sliders[1]
, 'v2':input_sliders[2]
, 'v3':input_sliders[3]
, 'v4':input_sliders[4]
, 'v5':input_sliders[5]
, 'v6':input_sliders[6]
, 'r1':input_radios
, 'o0':onetime_sliders[0]
, 'o1':onetime_sliders[1]
, 'o2':onetime_sliders[2]
, 'o3':onetime_sliders[3]
, 'o4':onetime_sliders[4]
, 'o5':onetime_sliders[5]
, 'o6':onetime_sliders[6]
, 'o7':onetime_sliders[7]
, 'o8':onetime_sliders[8]
, 'o9':onetime_sliders[9]
, 'o10':onetime_sliders[10]
, 'o11':onetime_sliders[11]
, 'o12':onetime_sliders[12]
, 'l0':lifetime_sliders[0]
, 'l1':lifetime_sliders[1]
, 'l2':lifetime_sliders[2]
, 'l3':lifetime_sliders[3]
})
interactive_plot2= widgets.interactive_output(show_output2, {
 'v0':input_sliders[0]
, 'v1':input_sliders[1]
, 'v2':input_sliders[2]
, 'v3':input_sliders[3]
, 'v4':input_sliders[4]
, 'v5':input_sliders[5]
, 'v6':input_sliders[6]
})





# Mortgage influencers

In [10]:
for input_slider in input_sliders:
    display(input_slider)

FloatSlider(value=3.0, description='Intrest Rate (%/yr)', layout=Layout(width='95%'), max=10.0, min=1.0, step=…

IntSlider(value=300, description='House Price ($k)', layout=Layout(width='95%'), max=1000, min=100, style=Slid…

IntSlider(value=60, description='Down Payment ($k)', layout=Layout(width='95%'), min=1, style=SliderStyle(desc…

IntSlider(value=30, description='Term (years)', layout=Layout(width='95%'), max=60, min=1, style=SliderStyle(d…

FloatSlider(value=2.0, description='Private Mortgage Insurance (%/yr)', layout=Layout(width='95%'), max=5.0, s…

FloatSlider(value=1.0, description='Payment Multiplier (%)', layout=Layout(width='95%'), max=5.0, min=1.0, sty…

IntSlider(value=1500, description='Specific Payment Amount ($)', layout=Layout(width='95%'), max=5000, min=500…

# One-time amounts

In [11]:
for onetime_slider in onetime_sliders[:3]:
    display(onetime_slider)

FloatSlider(value=2.0, description='Closing Costs (%)', layout=Layout(width='95%'), max=5.0, min=1.0, style=Sl…

IntSlider(value=350, description='Appraisal Fee ($)', layout=Layout(width='95%'), max=400, min=300, style=Slid…

IntSlider(value=400, description='Inspection Fee ($)', layout=Layout(width='95%'), max=500, min=300, style=Sli…

# Home repairs

In [12]:
for onetime_slider in onetime_sliders[3:]:
    display(onetime_slider)

IntSlider(value=0, description='Paint ($)', layout=Layout(width='95%'), max=500, style=SliderStyle(description…

IntSlider(value=0, description='Cabinets ($)', layout=Layout(width='95%'), max=500, style=SliderStyle(descript…

IntSlider(value=0, description='Appliances ($)', layout=Layout(width='95%'), max=500, style=SliderStyle(descri…

IntSlider(value=0, description='Furniture ($)', layout=Layout(width='95%'), max=500, style=SliderStyle(descrip…

IntSlider(value=0, description='Windows ($)', layout=Layout(width='95%'), max=500, style=SliderStyle(descripti…

IntSlider(value=0, description='Roof ($)', layout=Layout(width='95%'), max=500, style=SliderStyle(description_…

IntSlider(value=0, description='Tile ($)', layout=Layout(width='95%'), max=500, style=SliderStyle(description_…

IntSlider(value=0, description='Carpet ($)', layout=Layout(width='95%'), max=500, style=SliderStyle(descriptio…

IntSlider(value=0, description='Lawn ($)', layout=Layout(width='95%'), max=500, style=SliderStyle(description_…

IntSlider(value=0, description='Doors ($)', layout=Layout(width='95%'), max=500, style=SliderStyle(description…

# Lifetime amounts

In [13]:
for lifetime_slider in lifetime_sliders:
    display(lifetime_slider)

IntSlider(value=600, description='Home Insurance ($/yr)', layout=Layout(width='95%'), max=5000, min=500, style…

IntSlider(value=200, description='HOA contribution ($/mo)', layout=Layout(width='95%'), max=700, style=SliderS…

FloatSlider(value=4.5, description='Property Taxes ($k/yr)', layout=Layout(width='95%'), max=200.0, style=Slid…

IntSlider(value=300, description='Utilities ($/mo)', layout=Layout(width='95%'), max=1000, style=SliderStyle(d…

# Output

In [14]:
display(interactive_plot1)

Output()

# How to reduce spending

In [15]:
display(interactive_plot2)

Output()